## datasets

- `DatasetDict`
    - `Dataset`
        - 可支持索引

In [2]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos", "plus")

Using the latest cached version of the module from /home/whaow/.cache/huggingface/modules/datasets_modules/datasets/clinc_oos/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1 (last modified on Wed Jun 14 00:03:25 2023) since it couldn't be found locally at clinc_oos., or remotely on the Hugging Face Hub.
Found cached dataset clinc_oos (/home/whaow/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
type(clinc)

datasets.dataset_dict.DatasetDict

In [30]:
clinc

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})

In [4]:
clinc['train']

Dataset({
    features: ['text', 'intent'],
    num_rows: 15250
})

In [5]:
clinc['train'][42]

{'text': 'what phrase means goodbye in hawaii', 'intent': 61}

In [17]:
# clinc['train'].features

In [14]:
clinc['train'].features['intent'].int2str(61)

'translate'

In [1]:
intents = clinc['train'].features['intent']
intents.num_classes

NameError: name 'clinc' is not defined

In [29]:
# 维护类别id及其语义信息
# intents = clinc['train'].features['intent']
# intents = clinc['test'].features['intent']
intents = clinc['validation'].features['intent']
print(intents.int2str(61))
print(intents.str2int(intents.int2str(61)))

translate
61


### map

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [3]:
clinc

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})

In [7]:
clinc.items()

dict_items([('train', Dataset({
    features: ['text', 'intent'],
    num_rows: 15250
})), ('validation', Dataset({
    features: ['text', 'intent'],
    num_rows: 3100
})), ('test', Dataset({
    features: ['text', 'intent'],
    num_rows: 5500
}))])

In [9]:
type(clinc['train'])

datasets.arrow_dataset.Dataset

In [11]:
clinc.map??

In [19]:
# batched 默认为 Fasle，batch_size 默认为 1000
# 也就是什么都不指定时，默认的处理为一次处理一个样本；
# batched == False 时，默认为 batch_size = 1000
clinc_enc = clinc.map(lambda batch: tokenizer(batch['text'], truncation=True), 
                      batched=True, 
#                       batch_size=1000,
                      remove_columns=["text"])

Loading cached processed dataset at /home/whaow/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1/cache-7f8f509bccf0bd4e.arrow
Loading cached processed dataset at /home/whaow/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1/cache-cc1b9212398c2167.arrow
Loading cached processed dataset at /home/whaow/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1/cache-5bcc0db64fa44b4e.arrow


## metrics

In [18]:
from datasets import load_metric

In [20]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [21]:
accuracy_score = load_metric('accuracy')


In [32]:
# accuracy_score
# references: true labels;
accuracy_score.compute(predictions=[1, 1, 2, 2], references=[2, 1, 2, 1])

{'accuracy': 0.5}

In [ ]:

# from datasets import load_metric
# accuracy_score = load_metric('accuracy')
import evaluate
accuracy_score = evaluate.load('accuracy')


### glue metric

> General Language Understanding Evaluation (GLUE) benchmark

The **M**icrosoft **R**esearch **P**araphrase **C**orpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

In [ ]:
mrpc_metric = load_metric("glue", "mrpc")
sst2_metric = load_metric('glue', 'sst2') 

In [22]:
# import evaluate
# evaluate.load('glue', 'mrpc')
# evaluate.load('glue', 'sst2')

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

## models

In [34]:
# model.state_dict()
# model.state_dict().items()

# (key, value)
# list(model.state_dict().items())[42]